In [1]:
## object detection loop

# some packages need async support in jupyter
import nest_asyncio
from mb_utils.src.logging import logger

nest_asyncio.apply()

In [2]:
from mb_pytorch.dataloader.loader import DataLoader
from mb_pytorch.utils.version import version

In [3]:
version

'1.0.40'

In [4]:
from mb_pytorch.detection.training import detection_train_loop

In [18]:
yaml_file = '../scripts/detection/object_detection.yaml'

In [19]:
#k = DataLoader(yaml_file,logger=logger)
k = DataLoader(yaml_file)
k_data = k.data_dict['data']
model_data_dict = k.data_dict['model']

In [20]:
k.data_dict['model']

{'model_type': 'detection',
 'model': 'faster_rcnn',
 'use_pretrained': True,
 'use_own_model': False,
 'model_path': '/home/malav/Desktop/mb_packages/mb_pytorch/models/...',
 'model_layer': 'avgpool',
 'load_model': False,
 'use_torchvision_models': True,
 'model_name': 'fasterrcnn_resnet50_fpn',
 'model_version': '50',
 'model_optimizer': 'Adam',
 'model_loss': 'cross_entropy',
 'model_scheduler': 'StepLR',
 'model_pretrained': True,
 'model_epochs': 10,
 'model_num_classes': 41,
 'model_initialization': 'None',
 'model_train_parameters': {'Adam': {'lr': 0.001,
   'betas': [0.9, 0.999],
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False},
  'SGD': {'lr': 0.01,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False},
  'RMSprop': {'lr': 0.01,
   'alpha': 0.99,
   'eps': 1e-08,
   'weight_decay': 0,
   'momentum': 0,
   'centered': False},
  'cross_entropy': {'weight': 'None',
   'size_average': 'None',
   'ignore_index': -100,
   'reduce': 'None',
  

In [21]:
k.data_load()

Files already downloaded and verified
Files already downloaded and verified


(<torch.utils.data.dataloader.DataLoader at 0x7424b1329790>,
 Dataset CelebA
     Number of datapoints: 162770
     Root location: /home/malav/Desktop/mb_packages/mb_pytorch/data/
     Target type: ['attr', 'bbox']
     Split: train
     StandardTransform
 Transform: <mb_pytorch.dataloader.loader.JointTransforms object at 0x7424a68f5750>,
 Dataset CelebA
     Number of datapoints: 19962
     Root location: /home/malav/Desktop/mb_packages/mb_pytorch/data/
     Target type: ['attr', 'bbox']
     Split: test
     StandardTransform
 Transform: <mb_pytorch.dataloader.loader.JointTransforms object at 0x7424ef788650>)

In [22]:
from mb_pytorch.models.modelloader import ModelLoader,ModelExtractor

model_data_load = ModelLoader(k.data_dict['model'],logger=logger)

model_data =  model_data_load.get_model()
#model_data_ex = ModelExtractor(model_data)

train_loader,val_loader,_,_ = k.data_load()

all_data = k.data_dict

/home/malav/Desktop/mb_packages/mb_pytorch/env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/malav/Desktop/mb_packages/mb_pytorch/env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Fri 16:28:15,762 INF Model fasterrcnn_resnet50_fpn loaded from torchvision.models.                                                   


Files already downloaded and verified
Files already downloaded and verified


In [23]:
print(len(train_loader.dataset.attr_names))
train_loader.dataset.attr_names[:3]

41


['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive']

In [24]:
from mb_pytorch.training.train_params import train_helper

model_yaml_data = k.data_dict['model']
loss_attr,optimizer_attr,optimizer_dict,scheduler_attr,scheduler_dict = train_helper(model_yaml_data) 

In [25]:
print(loss_attr)
print(optimizer_attr)
print(optimizer_dict)
print(scheduler_attr)
print(scheduler_dict)

<function cross_entropy at 0x7425021b9080>
<class 'torch.optim.adam.Adam'>
{'lr': 0.001, 'betas': [0.9, 0.999], 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}
<class 'torch.optim.lr_scheduler.StepLR'>
{'step_size': 3, 'gamma': 0.2}


In [26]:

from torch.utils.tensorboard import SummaryWriter
import os
import torch
# ## setting the trianing params

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# optimizer = optimizer_attr(model_data.parameters(),**optimizer_dict)
# scheduler = scheduler_attr(optimizer,**scheduler_dict)

# best_val_loss = float('inf')

path_logs = os.path.join(all_data['data']['datasets_params_train']['root'], 'logs')
writer = SummaryWriter(log_dir=path_logs)

# all_model_data= k.data_dict['model']

In [27]:
from mb_pytorch.detection.training import detection_train_loop

detection_train_loop(k,scheduler=None,writer=writer,logger=logger,gradcam=None,device=device)

Fri 16:28:32,934 INF Training loop Starting                                                                                          
Fri 16:28:33,387 INF Model fasterrcnn_resnet50_fpn loaded from torchvision.models.                                                   
Fri 16:28:33,389 INF Model Loaded                                                                                                    


Files already downloaded and verified
Files already downloaded and verified


Fri 16:28:50,603 INF Optimizer and Scheduler Loaded                                                                                  
Fri 16:28:50,605 INF Loss: <function cross_entropy at 0x7425021b9080>                                                                
Fri 16:28:50,607 INF Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: [0.9, 0.999]
    capturable: False
    differe
Fri 16:28:50,609 INF Scheduler: None                                                                                                 
  0%|          | 0/10 [00:00<?, ?it/s]


AttributeError: 'Tensor' object has no attribute 'items'

In [17]:
model_data

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [37]:
for idx, (images, targets) in enumerate(train_loader):
    print(images.shape)
    print(targets)
    break

torch.Size([4, 3, 218, 178])
tensor([[ 95,  71, 226, 313],
        [ 72,  94, 221, 306],
        [216,  59,  91, 126],
        [622, 257, 564, 781]])


In [17]:
k.trainloader.dataset.attr_names[:3] ## need to add labels in the dataloader

['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive']

In [14]:
k.trainloader.dataset.attr[:3]

tensor([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
         1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
         1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]])